[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elsamuel27/snapthon/blob/main/Snapthon.ipynb)

In [ ]:
#@title # Paso 1: Rellena el formulario a continuación y ejecuta la celda

mult = "Video" #@param ["Video", "Playlist"]
link = "" #@param {type: "string"}
ft = "Video" #@param ["Video", "Audio"]
format = "mp4" #@param ["mp4", "mp3"] {allow-input: true}
route = "/content/Descargados"

# Paso 2: Ejecuta las celdas ocultas y espera

In [ ]:
#@title Actualizar pip

!pip install -U pip

In [ ]:
#@title Instalaciones

!pip install pytube
!sudo apt install zip -y

In [ ]:
#@title Importaciones

from pytube import YouTube
from pytube import Playlist
from google.colab import files

In [ ]:
#@title Descargas en el entorno

characters = ["\\", "/", ":", "*", "?", "\"", "<", ">", "|"]

if mult == "Video":
    yt = YouTube(link)
    title = yt.title
    for c in characters:
      title = title.replace(c, "")
    file = f"{title}.{format}"

    if ft == "Video":
        yd = yt.streams.get_highest_resolution()

        yd.download(route, file)
        print("¡Vídeo descargado! Hasta luego :D")
    elif ft == "Audio":
        yd = yt.streams.get_audio_only()

        yd.download(route, file)
        print("¡Vídeo descargado! Hasta luego :D")
elif mult == "Playlist":
    pl = Playlist(link)
    if ft == "Video":
        for video in pl.videos:
            title = video.title
            for c in characters:
              title = title.replace(c, "")
            file = f"{title}.{format}"
            try:
                yd = video.streams.get_highest_resolution()
                yd.download(route, file)
                print("+1 vídeo en la carpeta")
            except:
                print("No se puede acceder a los datos de este vídeo. Saltamos al siguiente")
    elif ft == "Audio":
        for video in pl.videos:
            title = video.title
            for c in characters:
                title = title.replace(c, "")
            try:
                yd = video.streams.get_audio_only()
                yd.download(route, f"{title}.{format}")
                print("+1 vídeo en la carpeta")
            except:
                print("No se puede acceder a los datos de este vídeo. Saltamos al siguiente")

# Paso 3: Elige la opción que más te convenga

Recomendada la primera opción para listas con muchos vídeos, la segunda para unos pocos o directamente uno

In [ ]:
#@title Archivo ZIP

!zip -r /content/Descargados.zip Descargados
!rm -r Descargados

files.download("/content/Descargados.zip")

In [ ]:
#@title Archivo/s suelto/s

import os

for video in os.listdir("/content/Descargados"):
  files.download(f"/content/Descargados/{video}")

# Paso 4: Disfruta el contenido descargado